# openAI API를 통해 chatGPT에게 질문을 보내고 respond를 저장하고, 예외를 처리합니다.
- 질문
    - [질문 내용] + [문자 내용] 
        - 질문 내용
            - 예를 들어 "이것이 스팸 문자일까? 아니면 햄일까?" 와 같은 식의 텍스트 입니다.
            - 다음과 같은 형태의 입력을 받도록 작성하였습니다.

        - 문자 내용
            - KISA에서 받은 스팸 문자들입니다.
            - 다음과 같은 형태로 입력받도록 작성하였습니다.
                - KISA에서 제공하는 .csv 파일
                - 아직 엑셀 입력은 미구현

- openAI API는 병렬처리를 지원하지 않기 때문에 시간이 오래 걸립니다.

In [5]:
import os
import glob
import json
import time
import openai


from query_openai import queryQuestions
from text_message import TextMessage, parseKISASpamDataFile

# openAI API key를 이 파일에 저장하세요.
# [KEY1, KEY2, ... ] 와 같은 형식으로 저장하세요.
# 여기서는 1개의 키만 사용합니다.
OPENAI_API_KEY_FILE_PATH = "./openai_api_keys.json"

# chatGPT 모델 이름을 선택하세요
# https://platform.openai.com/docs/models/overview
GPT_MODEL_NAME = "gpt-3.5-turbo"

# KISA 에서 전달받은 스팸 문자 csv 파일을 이 디렉토리에 저장하세요
DATA_DIR_PATH = "./data/"
assert os.path.exists(DATA_DIR_PATH), f"{DATA_DIR_PATH} 디렉토리가 존재하지 않습니다."

# 결과를 저장할 디렉토리를 지정하세요
SAVE_FILE_DIR_PATH = "./results/"
assert os.path.exists(SAVE_FILE_DIR_PATH), f"{SAVE_FILE_DIR_PATH} 디렉토리가 존재하지 않습니다."

# chatGPT에 보낼 쿼리의 질문을 지정하세요
QUERY_QUESTION = "다음 문자는 스팸 문자일까, 아닐까? \n"

# read openAI API key
with open(OPENAI_API_KEY_FILE_PATH, "r") as fp :
    openai_api_key = json.load(fp)[0]
    # openai API 키 인증
    openai.api_key = openai_api_key

# read KISA-provided spam data file path 
spam_data_file_path_list = glob.glob(os.path.join(DATA_DIR_PATH, "*.csv"))
print("KISA 스팸 문자 데이터 파일 목록 :")
for idx, file_path in enumerate(spam_data_file_path_list) :
    print(f"{idx}\t {file_path}")
print()

spam_data_file_idx = input("처리할 스팸 문자 데이터 파일의 인덱스를 입력하세요 : ")
print()
SPAM_DATA_PATH = spam_data_file_path_list[int(spam_data_file_idx)]

print("처리할 파일 :")
print(SPAM_DATA_PATH)

KISA 스팸 문자 데이터 파일 목록 :
0	 ./data\20220830_GBL.csv
1	 ./data\20220911_GBL.csv
2	 ./data\20220918_GBL.csv
3	 ./data\20220925_GBL.csv
4	 ./data\20221002_GBL.csv
5	 ./data\20221009_GBL.csv
6	 ./data\20221016_GBL.csv
7	 ./data\20221023_GBL.csv
8	 ./data\20221030_GBL.csv
9	 ./data\20221106_GBL.csv
10	 ./data\20221113_GBL.csv
11	 ./data\20221120_GBL.csv
12	 ./data\20221127_GBL.csv
13	 ./data\20221204_GBL.csv
14	 ./data\20221211_GBL.csv
15	 ./data\20221218_GBL.csv
16	 ./data\20221225_GBL.csv
17	 ./data\20230101_GBL_copy.csv
18	 ./data\20230101_GBL_copy2.csv
19	 ./data\20230108_GBL_copy3.csv


처리할 파일 :
./data\20221225_GBL.csv


In [6]:
spam_data_list = parseKISASpamDataFile(SPAM_DATA_PATH)
for data in spam_data_list :
    data.query_content = QUERY_QUESTION + data.content

: 

### ChatGPT 쿼리 
- queryQuestions 함수는 spam_data_list 중 query_succeed==False 인 원소만 골라내 쿼리를 날립니다.
- 아래의 셀에서 number of data to process 가 0 이 될 때 까지 반복적으로 실행하십시오

In [8]:
queryQuestions(
    model_name = GPT_MODEL_NAME,
    spam_data_list = spam_data_list
)

number of data to process :  10
processing..


100%|██████████| 10/10 [01:08<00:00,  6.86s/it]

[0] data left to process


## save result to file
- 모든 결과를 json 파일에 저장합니다.

In [9]:
RESULT_FILE_NAME = "{}_processed_{}.json".format(
    os.path.basename(SPAM_DATA_PATH).split(".")[0],
   time.strftime("%Y%m%d%H%M%S")
)
RESULT_FILE_PATH = os.path.join(SAVE_FILE_DIR_PATH, RESULT_FILE_NAME)

print("saving result to {}".format(RESULT_FILE_PATH))
with open(RESULT_FILE_PATH, "w") as fp :
    json.dump(
        list(map(
            lambda data : data.toDict(),
            spam_data_list
        )),
        fp,
        indent=2,
        ensure_ascii=False
    )
print("file saved")

saving result to 20220830_GBL_processed_20230830181245.json
file saved
